# Extraction des informations PDF du dossier FTs

Ce notebook extrait pour **chaque PDF** du dossier `FTs` :
- **ISIN**
- **Issuer** (détecté parmi la liste connue)
- **Nombre d'occurrences** de chaque mot choisi (insensible à la casse et aux accents)

Si une étape échoue (ex. issuer non détecté), les autres champs (ISIN, comptages) sont quand même remplis.

Modifiez la liste `MOTS_A_COMPTER` ci-dessous pour choisir les mots à compter.

In [ ]:
import re
import unicodedata
from pathlib import Path
import pandas as pd
import pdfplumber

# --- Liste des issuers reconnus (ordre longueur décroissante pour éviter sous-chaînes) ---
LISTE_ISSUERS = [
    "Merrill Lynch", "Bank of America", "Morgan Stanley", "Société Générale",
    "Societe Generale", "Societe General", "Deutsche Bank", "Deutsch Bank",
    "Crédit Agricole", "Credit Agricole", "UniCredit", "uni credit",
    "BNP Paribas", "Citigroup", "Leonteq", "Santander", "Natixis", "Barclays",
    "JPMorgan", "J.M. Morgan", "Goldman Sachs", "Goldman", "HSBC", "Nomura",
    "UBS", "BOFA", "Citi", "RBC",
]
ISSUER_CANONIQUE = {
    "bofa": "Bank of America", "bank of america": "Bank of America", "merrill lynch": "Bank of America",
    "citi": "Citigroup", "citigroup": "Citigroup",
    "société générale": "Société Générale", "societe generale": "Société Générale", "societe general": "Société Générale",
    "deutsche bank": "Deutsche Bank", "deutsch bank": "Deutsche Bank",
    "unicredit": "UniCredit", "uni credit": "UniCredit",
    "jpmorgan": "JPMorgan", "j.m. morgan": "JPMorgan",
    "goldman": "Goldman Sachs", "goldman sachs": "Goldman Sachs",
}

def normaliser(texte):
    if not texte:
        return ""
    nfd = unicodedata.normalize("NFD", texte.lower())
    return "".join(c for c in nfd if unicodedata.category(c) != "Mn")

def extraire_texte_pdf(chemin):
    blocs = []
    with pdfplumber.open(chemin) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                blocs.append(t)
    return "\n".join(blocs)

def extraire_isin(texte):
    m = re.search(r"\b([A-Z]{2}[A-Z0-9]{9}[0-9])\b", texte, re.IGNORECASE)
    return m.group(1).upper() if m else None

def extraire_issuer(texte):
    texte_n = normaliser(texte)
    ordered = sorted(LISTE_ISSUERS, key=len, reverse=True)
    for nom in ordered:
        nom_n = normaliser(nom)
        if nom_n and nom_n in texte_n:
            return ISSUER_CANONIQUE.get(nom_n, nom)
    return None

def compter_mot(texte, mot_test):
    if not texte or not mot_test:
        return 0
    texte_n = normaliser(texte)
    mot_n = normaliser(mot_test)
    motif = r"\b" + re.escape(mot_n) + r"\b"
    return len(re.findall(motif, texte_n))

def _cle_nb(mot):
    return "nb_" + mot.replace(" ", "_")

def extraire_infos_pdf(chemin_pdf, mots_test=None):
    if mots_test is None:
        mots_test = ["mot_test"]
    if isinstance(mots_test, str):
        mots_test = [mots_test]
    chemin = Path(chemin_pdf)
    if not chemin.exists():
        raise FileNotFoundError(f"PDF introuvable: {chemin}")
    try:
        texte = extraire_texte_pdf(chemin)
    except Exception as e:
        out = {"isin": None, "issuer": None, "erreur": str(e)}
        for mot in mots_test:
            out[_cle_nb(mot)] = None
        return out
    try:
        isin = extraire_isin(texte)
    except Exception:
        isin = None
    try:
        issuer = extraire_issuer(texte)
    except Exception:
        issuer = None
    out = {"isin": isin, "issuer": issuer}
    for mot in mots_test:
        try:
            out[_cle_nb(mot)] = compter_mot(texte, mot)
        except Exception:
            out[_cle_nb(mot)] = None
    return out

# --- Paramètres (dossiers et mots à compter) ---
DOSSIER_FTS = Path("FTs")
DOSSIER_RESULTATS = Path("Extraction Result")
MOTS_A_COMPTER = ["risque", "crédit"]

In [ ]:
# Liste de tous les PDF dans FTs
fichiers_pdf = sorted(DOSSIER_FTS.glob("*.pdf"))
print(f"{len(fichiers_pdf)} fichier(s) PDF trouvé(s) dans {DOSSIER_FTS}")
for f in fichiers_pdf:
    print(f"  - {f.name}")

In [ ]:
# Extraction des infos pour chaque PDF (une erreur sur une étape ne bloque pas les autres)
resultats = []
for chemin in fichiers_pdf:
    try:
        infos = extraire_infos_pdf(chemin, mots_test=MOTS_A_COMPTER)
        ligne = {"fichier": chemin.name, "isin": infos["isin"], "issuer": infos["issuer"]}
        for mot in MOTS_A_COMPTER:
            cle = f"nb_{mot.replace(' ', '_')}"
            ligne[cle] = infos.get(cle)
        if "erreur" in infos:
            ligne["erreur"] = infos["erreur"]
        resultats.append(ligne)
    except Exception as e:
        ligne = {"fichier": chemin.name, "isin": None, "issuer": None, "erreur": str(e)}
        for mot in MOTS_A_COMPTER:
            ligne[f"nb_{mot.replace(' ', '_')}"] = None
        resultats.append(ligne)

df = pd.DataFrame(resultats)
df

In [ ]:
# Export Excel dans le dossier "Extraction Result"
DOSSIER_RESULTATS.mkdir(parents=True, exist_ok=True)
fichier_excel = DOSSIER_RESULTATS / "extraction_FTs.xlsx"
df.to_excel(fichier_excel, index=False, engine="openpyxl")
print(f"Résultats exportés dans {fichier_excel}")